# Artificial Intelligence 24/25 Assignment - Adversarial search strategies and Decision Trees

### Imports iniciais

In [88]:
import random
import time
import pandas as pd
import numpy as np
import csv

from copy import deepcopy
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### `ConnectFourState`

  * Representa o estado do tabuleiro (6×7) e o jogador ativo (`1`=X, `-1`=O).
  * Métodos:

    * `__init__(board=None, player=1)`: inicializa tabuleiro vazio ou a partir de uma cópia.
    * `clone()`: devolve uma cópia profunda do estado.
    * `get_legal_moves()`: lista as colunas disponíveis.
    * `play_move(col)`: coloca a peça na coluna e alterna o jogador.
    * `check_win()`: devolve `1`/`-1` se houver vitória, `0` em caso de empate, ou `None` se o jogo ainda não tiver terminado.
* `print_board(state)`: imprime o tabuleiro no terminal.

In [89]:
# Classe que define o tabuleiro, as jogadas e os critérios de vitória do jogo Connect Four
class ConnectFourState:
    ROWS = 6
    COLS = 7

    def __init__(self, board=None, player=1):
        # 0 = vazio, player 1 = X, player -1 = O
        self.board = deepcopy(board) if board is not None else [[0]*self.COLS for _ in range(self.ROWS)]
        self.player = player

    def clone(self):
        return ConnectFourState(self.board, self.player)

    def get_legal_moves(self):
        return [c for c in range(self.COLS) if self.board[0][c] == 0]

    def play_move(self, col):
        '''Executa jogada na coluna indicada, retornando True se a jogada for válida.'''
        for r in range(self.ROWS-1, -1, -1):
            if self.board[r][col] == 0:
                self.board[r][col] = self.player
                self.player *= -1
                return True
        return False

    def check_win(self):
        '''Deteta vitória se o resultado for 1/-1, empate se for 0 ou None se o jogo ainda não tiver terminado.'''
        directions = [(0,1),(1,0),(1,1),(1,-1)]
        for r in range(self.ROWS):
            for c in range(self.COLS):
                p = self.board[r][c]
                if p == 0:
                    continue
                for dr, dc in directions:
                    cnt, rr, cc = 0, r, c
                    for _ in range(4):
                        if 0 <= rr < self.ROWS and 0 <= cc < self.COLS and self.board[rr][cc] == p:
                            cnt += 1; rr += dr; cc += dc
                        else:
                            break
                    if cnt == 4:
                        return p
        if not self.get_legal_moves():
            return 0
        return None

# Impressão do tabuleiro
def print_board(state):
    print('\n  ' + ' '.join(str(c) for c in range(state.COLS)))
    for row in state.board:
        print(' |' + ' '.join('X' if v==1 else 'O' if v==-1 else '.' for v in row) + '|')
    print()

# transformar o tabuleiro (`6×7` matriz) e o jogador atual num vetor unidimensional, para posteriormente alimentar a árvore ID3. 
def state_to_series(state, feature_columns):
    vec = []
    for row in state.board:
        vec.extend(row)
    vec.append(state.player)
    return pd.Series(vec, index=feature_columns)


### Monte Carlo Tree Search (MCTS)

* Algoritmo que aplica UCT na raiz para distribuir simulações entre movimentos iniciais utilizando a função: `mcts(root_state, iter_limit=1000, c=1.4)`. 
* Esta função: 
    * Inicializa estatísticas `{w, n}` para cada jogada legal tendo em conta o limite máximo de iterações dado como argumento.
    * `total_n =` soma de todas as `n`  
    * Para cada jogada, calcula UCT com a formula: $ \mathrm{UCT}_i \;=\; \frac{w_i}{n_i} \;+\; C_p \,\sqrt{\frac{\ln N}{n_i}} $
    * Seleciona a jogada com maior UCT  
    * Clona o estado, aplica a jogada e executa playout atá encontrar um estado final
    * Atualiza `n += 1` e `w += 1` se encontrar uma vitória do jogador dado como argumento  
    * Retorna a jogada com maior número de visitas (`n`)

In [90]:
import math, random

def mcts(root_state, iter_limit=1000, c=1.4):
    # estatísticas por movimento: w = vitórias, n = simulações
    moves = root_state.get_legal_moves()
    stats = {mv: {'w': 0, 'n': 0} for mv in moves}

    def rollout(state):
        # playout aleatório até ao fim do jogo
        winner = state.check_win()
        while winner is None:
            mv = random.choice(state.get_legal_moves())
            state.play_move(mv)
            winner = state.check_win()
        return winner

    for _ in range(iter_limit):
        total_n = sum(stats[m]['n'] for m in moves)

        def uct_score(mv):
            w, n = stats[mv]['w'], stats[mv]['n']
            if n == 0:
                return float('inf')
            return (w / n) + c * math.sqrt(math.log(total_n) / n)

        # seleção da jogada segundo UCT
        best_mv = max(moves, key=uct_score)

        # playout a partir do movimento selecionado
        sim = root_state.clone()
        sim.play_move(best_mv)
        winner = rollout(sim)

        # atualização das estatísticas
        stats[best_mv]['n'] += 1
        if winner == root_state.player:
            stats[best_mv]['w'] += 1

    # retorna o estado com maior percentagem de vitórias
    return max(moves, key=lambda mv: stats[mv]['n'])


### Modos de jogo

1. `PvP`: jogador vs jogador
2. `PvC`: jogador vs MCTS
3. `CvC`: MCTS vs ID3

In [91]:
# Loop principal
def play_game():
    print("=== Connect-Four ===")
    print("Modo: 1) PvP  2) PvC MCTS  3) CvC ID3 vs MCTS")
    mode = input("Escolha: ").strip()
    while mode not in ('1','2','3'):
        mode = input("Escolha: ").strip()

    if mode == '3':
        print("\nA treinar árvore de decisão (ID3)...")
        df = pd.read_csv("connect4_dataset.csv")
        
        feature_columns = df.columns[:-1]
        X = df[feature_columns]
        y = df.iloc[:, -1]
        max_depth = int(np.log2(len(df)) + 1)
        training_data = pd.concat([X, y], axis=1)
        decision_tree = buildTree(training_data, max_depth)
        print("Árvore treinada!\n")

    state = ConnectFourState()
    print_board(state)

    while True:
        if mode == '1' or (mode == '2' and state.player == 1):
            # PvP
            col = int(input(f"Jogador {'1 (X)' if state.player==1 else '2 (O)'}, escolha coluna {state.get_legal_moves()}: "))

        elif mode == '2' and state.player == -1:
            # PvC MCTS
            print("Computador (MCTS) a pensar...")
            start = time.time()
            col = mcts(state)
            print(f"Tempo de cálculo: {time.time() - start:.2f}s")

        else:
            # CvC
            if state.player == 1:
                print("Computador (ID3) a pensar...")
                example = state_to_series(state, feature_columns)
                col = int(classifyExample(example, decision_tree))
            else:
                print("Computador (MCTS) a pensar...")
                start = time.time()
                col = mcts(state)
                print(f"Tempo de cálculo: {time.time() - start:.2f}s")

        state.play_move(col)
        print_board(state)
        res = state.check_win()
        if res is not None:
            if res == 0:
                print("Empate!")
            else:
                w = '1 (X)' if res == 1 else '2 (O)'
                print(f"Vitória do jogador {w}!")
            break

### Dataset Connect Four

* Para treinar o ID3 queremos criar pares `(estado, melhor_jogada)`. Para isso, seguimos os seguintes passos:

  1. Começa com um tabuleiro vazio.
  2. Avançamos um número aleatório de jogadas (até 20).
  3. Se não for um nó terminal, usar MCTS para escolher a melhor jogada.
  4. Guardar o estado "flatten" + jogador ativo + label (coluna escolhida).


In [92]:
def state_to_feature_vector(state):
    """
    Transforma o estado de jogo num vetor de features:
    - flatten(board) com valores em {-1,0,1}
    - jogador a mover (1 ou -1)
    """
    features = []
    for row in state.board:
        features.extend(row)
    features.append(state.player)
    return features

def generate_connect_four_dataset(num_samples: int,
                                  playouts: int,
                                  filename: str):

    header = [f"cell_{i}" for i in range(ConnectFourState.ROWS * ConnectFourState.COLS)] + ["player", "move"]
    
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        
        for _ in tqdm(range(num_samples), desc="Gerar dataset"):
            state = ConnectFourState()
            
            # 1. Avança um número aleatório de jogadas
            depth = random.randint(0, 20)
            for _ in range(depth):
                legal = state.get_legal_moves()
                if not legal or state.check_win() is not None:
                    break
                state.play_move(random.choice(legal))
            
            # 2. Ignora os estados terminais
            if state.check_win() is not None:
                continue
            
            # 3. Calcula melhor jogada com Monte Carlo
            best_move = pure_monte_carlo_choice(state, playouts)
            
            # 4. Extrai vetor de features e escreve no ficheiro .csv (features + label)
            features = state_to_feature_vector(state)
            writer.writerow(features + [best_move])
    
    print(f"Dataset guardado em: {filename}")

# Para gerar o dataset, corremos o código, descomentando o trecho seguinte:

'''
if __name__ == "__main__":
    # Exemplo: 5000 amostras, 200 simulações por jogada
    generate_connect_four_dataset(num_samples=5000,
                                  playouts=200,
                                  filename="connect4_dataset.csv")
'''


'\nif __name__ == "__main__":\n    # Exemplo: 5000 amostras, 200 simulações por jogada\n    generate_connect_four_dataset(num_samples=5000,\n                                  playouts=200,\n                                  filename="connect4_dataset.csv")\n'

### Classe DeciTree (ID3)

Esta classe implementa o algoritmo ID3 para construir árvores de decisão de forma manual. 

Componentes principais:

- `Node`: estrutura de nó da árvore.  
  - `attribute`: atributo usado para divisão no nó.  
  - `value`: valor de corte ou categoria do atributo.  
  - `results`: classe alvo se for nó folha.  
  - `branches`: dicionário com ramos `"true_branch"` e `"false_branch"`.  
  - `counter`: número de exemplos neste nó.  

- `entropy(data)`: calcula a entropia da coluna alvo (última coluna).  
- `splitData(data, attribute, value)`: divide o DataFrame em dois ramos conforme valor.  
- `buildTree(data, max_depth, depth=0)`:  Verifica se encontra uma condição de paragem (atingiu um nó puro ou a profundidade máxima), para cada atributo e cada valor possível, divide os dados e calcula o ganho de informação selecionando a divisão com maior ganho de informação. Por fim cria recursivamente sub-árvores ou retorna o nó folha.  

- `classifyExample(example, tree)`: percorre a árvore dado um exemplo (`Series`) e retorna a classifição da classe.  
- `quartis(df)`: discretiza colunas contínuas em 4 intervalos usando `pd.qcut`.  


In [93]:
class Node:
    def __init__(self, attribute=None, value=None, results=None, branches=None, counter=None):
        self.attribute = attribute
        self.value = value
        self.results = results
        self.branches = branches if branches is not None else {}
        self.counter = counter

# Calcula a entropia da coluna alvo
def entropy(data):
    total = len(data)
    if total == 0:
        return 0
    counts = data.iloc[:, -1].value_counts()
    ent = 0
    for cnt in counts:
        p = cnt / total
        ent -= p * np.log2(p)
    return ent

# Divide os dados em dois ramos distintos
def splitData(data, attribute, value):
    true_branch = data[data[attribute] == value]
    false_branch = data[data[attribute] != value]
    return {"true_branch": true_branch, "false_branch": false_branch}

# Constrói a árvore de decisão
def buildTree(data, max_depth, depth=0):
    counter = len(data)
    if counter == 0:
        return Node(results=None, counter=0)
    if len(data.iloc[:, -1].unique()) == 1 or depth == max_depth:
        return Node(results=data.iloc[:, -1].mode()[0], counter=counter)

    current_entropy = entropy(data)
    best_gain = 0
    best_sets = None
    best_criteria = None

    for attribute in data.columns[:-1]:
        for val in data[attribute].unique():
            sets = splitData(data, attribute, val)
            t = len(sets["true_branch"]) / counter
            gain = current_entropy \
                   - t * entropy(sets["true_branch"]) \
                   - (1 - t) * entropy(sets["false_branch"])
            if gain > best_gain \
               and len(sets["true_branch"]) > 0 \
               and len(sets["false_branch"]) > 0:
                best_gain = gain
                best_sets = sets
                best_criteria = (attribute, val)

    if best_gain > 0:
        tb = buildTree(best_sets["true_branch"], max_depth, depth + 1)
        fb = buildTree(best_sets["false_branch"], max_depth, depth + 1)
        return Node(
            attribute=best_criteria[0],
            value=best_criteria[1],
            branches={"true_branch": tb, "false_branch": fb},
            counter=counter
        )
    else:
        return Node(results=data.iloc[:, -1].mode()[0], counter=counter)

# Classifica um dado exemplo, de acordo com a árvore de decisão gerada
def classifyExample(example, tree):
    if tree.results is not None:
        return tree.results
    branch = tree.branches["true_branch"] \
        if example[tree.attribute] == tree.value \
        else tree.branches["false_branch"]
    return classifyExample(example, branch)

# Divisão dos dados em 4 intervalos
def quartis(df):
    for col in df.columns[:-1]:
        if pd.api.types.is_numeric_dtype(df[col]):
            try:
                df[col] = pd.qcut(df[col], q=4, duplicates="drop")
            except:
                pass
    return df

### Árvore de decisão no dataset Iris

1. Carregar dados do dataset `iris.csv` (atributos: sepal/petal length/width, classe).
2. Discretizar, usando `quartis(df)` para converter valores contínuos.
3. Definir `max_depth`: `int(log2(n_exemplos))+1`, para prevenir overfitting.
4. Treinar o modelo `decision_tree = buildTree(df, max_depth)`.
5. Avaliar, fazendo uma separação dos dados entre treino e teste (neste caso: 70%/30%).
6. Classificar os exemplos e medir a sua accuracy.


In [94]:
# Carregar e discretizar
df = pd.read_csv("iris.csv")
df = quartis(df)

# Dividir treino/teste
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
train_df = pd.concat([X_train, y_train], axis=1)

# Definir profundidade máxima
max_depth = int(np.log2(len(train_df))) + 1

# Treinar a árvore
tree = buildTree(train_df, max_depth)

# Avaliar a árvore
y_pred = [classifyExample(row, tree) for _, row in X_test.iterrows()]
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9777777777777777


### Modo de jogo PvP

* Para simular um jogo entre dois jogadores, vamos assumir jogadas aleatórias.

In [ ]:
# contador de chamadas, para garantir que o primeiro argumento do input, é a escolha do modo de jogo
call_count = {"n": 0}
state = ConnectFourState()

def fake_input(prompt=None):
    call_count["n"] += 1
    if call_count["n"] == 1:
        choice = "1"
    else:
        # simula humano aleatório em PvP
        moves = state.get_legal_moves()
        choice = str(random.choice(moves))
    print(f"{prompt}{choice}")
    return choice

input = fake_input

play_game()

input = None


=== Connect-Four ===
Modo: 1) PvP  2) PvC MCTS  3) CvC ID3 vs MCTS
Escolha: 1

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|

Jogador 1 (X), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 1

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. X . . . . .|

Jogador 2 (O), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 0

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |O X . . . . .|

Jogador 1 (X), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 2

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |O X X . . . .|

Jogador 2 (O), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 3

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |O X X O . . .|

Jogador 1 (X), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 1

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . .

### Modo de jogo PvC (Player vs MCTS)

- Também simularemos as jogadas do jogador de forma aleatória.

In [ ]:
# contador de chamadas para que quando executado a primeira vez, ative o modo 2 (PvC)
call_count = {"n": 0}
state = ConnectFourState()  # necessário para saber jogadas possíveis no fake_input

def fake_input(prompt=None):
    call_count["n"] += 1
    if call_count["n"] == 1:
        choice = "2"
    else:
        moves = state.get_legal_moves()
        choice = str(random.choice(moves))
    print(f"{prompt}{choice}")
    return choice

# 1) Sobrescrever input()
input = fake_input

# 2) Chamar play_game
play_game()

input = None

=== Connect-Four ===
Modo: 1) PvP  2) PvC MCTS  3) CvC ID3 vs MCTS
Escolha: 2

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|

Jogador 1 (X), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 2

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . X . . . .|

Computador (MCTS) a pensar...
Tempo de cálculo: 0.40s

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . X O . . .|

Jogador 1 (X), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 6

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . X O . . X|

Computador (MCTS) a pensar...
Tempo de cálculo: 0.43s

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . O . . . .|
 |. . X O . . X|

Jogador 1 (X), escolha coluna [0, 1, 2, 3, 4, 5, 6]: 3

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .

### Modo de jogo MCTS vs ID3

* Carregar o dataset `connect4_dataset.csv` gerado pelo MCTS.  
* Selecionar colunas de características (todas menos `"move"`) e copiar para `train_df`.

In [ ]:
input = lambda _prompt: '3'

play_game()

input = None

=== Connect-Four ===
Modo: 1) PvP  2) PvC MCTS  3) CvC ID3 vs MCTS

A treinar árvore de decisão (ID3)...
Árvore treinada!


  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|

Computador (ID3) a pensar...

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . X . . .|

Computador (MCTS) a pensar...
Tempo de cálculo: 0.37s

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . X . O .|

Computador (ID3) a pensar...

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. X . X . O .|

Computador (MCTS) a pensar...
Tempo de cálculo: 0.33s

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. X O X . O .|

Computador (ID3) a pensar...

  0 1 2 3 4 5 6
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . .|
 |. . . . . . 